In [13]:
import chromadb
from chromadb.utils import embedding_functions
import sentence_transformers
import util
import numpy as np

In [3]:
Model = sentence_transformers.SentenceTransformer("E:\models\BAAI_bge-base-zh-v1.5").cuda().half()

In [7]:
encode_ef = Model.encode


In [30]:
embeddings = encode_ef([
    "你好啊，世界！",
    "世界，你好啊！",
    "水调歌头·明月几时有",
    "苏轼的词《水调歌头》全文",
    """
    丙辰中秋，欢饮达旦，大醉，作此篇，兼怀子由。 \n\n明月几时有？把酒问青天。不知天上宫阙，今夕是何年。我欲乘风归去，又恐琼楼玉宇，高处不胜寒。起舞弄清影，何似在人间？\n转朱阁，低绮户，照无眠。不应有恨，何事长向别时圆？人有悲欢离合，月有阴晴圆缺，此事古难全。但愿人长久，千里共婵娟。\n\n

    """,
    "但愿人长久，千里共婵娟"
                         
])

In [31]:
util.cosine_similarity(embeddings[5],embeddings[4])

0.6357

In [32]:
util.dot_product(embeddings[1],embeddings[5])

0.4219

In [16]:
np.linalg.norm(embeddings[1])

1.0

In [33]:
poets_df = util.read_csv("poets_3154.csv")